<a href="https://colab.research.google.com/github/shivamchoudhury06/Technocolabs_Internship/blob/main/Model_Accuracy_Technocolabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

**In this file we are Trying to get the accuracy and score of the model**

In [1]:
# #Importing Data Analysis Libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

import re
import pickle

# #Importing Natural Language Libraries
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec

# # Importing Keras for Word Embedding
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

#Importing libraries for Modeling 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


#For finding Accuracy
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report




In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cleaned_data.csv",nrows=10000)

In [ ]:
df.head()

,author,content,publish_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,1D_NICOLE_,#FoodPoisoning is not a joke! #Walmart #KochFa...,2015-11-26,48,40,394,RETWEET,Koch,0,1,Fearmonger
1,1D_NICOLE_,Thanks God that #turkey i ate few days ago was...,2015-11-26,48,40,395,RETWEET,Koch,0,1,Fearmonger
2,1D_NICOLE_,"who's got the worst make up ever, don't tell m...",2015-05-25,59,51,352,NO_ACTION,Koch,0,0,Fearmonger
3,1D_NICOLE_,"I hurt easy, I just don't show it. You can hur...",2015-05-25,59,51,354,NO_ACTION,Koch,0,0,Fearmonger
4,1D_NICOLE_,« The young are permanently in a state resembl...,2015-05-25,59,51,356,NO_ACTION,Koch,0,0,Fearmonger


In [ ]:
def dependent_column(a):
    if a == 'Fearmonger':
        return 1
    elif a == "RightTroll":
        return 2
    elif a == "LeftTroll":
        return 3
    elif a == 'NonEnglish':
        return 4
    elif a == 'Unknown':
        return 5
    elif a == 'HashtagGamer':
        return 6
    elif a == 'Commercial':
        return 7
    elif a == 'NewsFeed':
        return 8

In [ ]:
df["account_category_int"] = df.account_category.apply(dependent_column)

## Text Preprocessing
**1. Bog-of-Words**

**2. TF-IDF**




In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# 4. Removing Stopwords
stopwords=nltk.corpus.stopwords.words('english')
stopwords[1:5]

['me', 'my', 'myself', 'we']

In [ ]:
# Combining all the above stundents 
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df['content'].values):
    sentance = re.sub(r"http\S+", "", sentance)           # HTML Tags
    sentance = re.sub("\S*\d\S*", "", sentance).strip()   # To remove words like @care2 = " "
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.strip())

100%|██████████| 10000/10000 [00:00<00:00, 25761.37it/s]


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Using TF-IDF
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
count_vect = CountVectorizer()
vect=tf_idf_vect.fit_transform(preprocessed_text)

In [ ]:
#Using Bag-of-Words
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(preprocessed_text)
print("some feature names ", count_vect.get_feature_names()[:10])
print('='*50)

final_counts = count_vect.transform(preprocessed_text)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

some feature names  ['aa', 'aaaaaaaand', 'aaaaaand', 'aaaaand', 'aaand', 'aaarrrrrrrrrgggggg', 'aaawww', 'aaihs', 'aaliyahhaughton', 'aashaaa']
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (10000, 15035)
the number of unique words  15035


In [ ]:
#Splitting the data
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(vect, df['account_category_int'], test_size=0.33, random_state=42)
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(final_counts, df['account_category_int'], test_size=0.33, random_state=42)

**Finding Accuracy with Bag-of-Words**

In [ ]:
X_train_bow.shape, X_test_bow.shape, y_train_bow.shape, y_test_bow.shape

((6700, 15035), (3300, 15035), (6700,), (3300,))

In [ ]:
X_train_tfidf.shape, X_test_tfidf.shape, y_train_tfidf.shape, y_test_tfidf.shape

((6700, 1451), (3300, 1451), (6700,), (3300,))

In [ ]:
# Random Forest(BagofWords)
classifier =  RandomForestClassifier(n_estimators=50, verbose=True)
classifier.fit(X_train_bow, y_train_bow)
y_pred_bow = classifier.predict(X_test_bow)
report_bow = classification_report(y_test_bow, y_pred_bow)
acc = accuracy_score(y_test_bow,y_pred_bow) 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    8.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.2s finished
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report_bow)
print("Random Classifier Accuracy: ", acc*100,"%")

              precision    recall  f1-score   support

           1       1.00      0.46      0.63        48
           2       0.99      0.72      0.84       302
           3       0.87      0.98      0.92      2386
           4       0.00      0.00      0.00         1
           5       0.11      0.06      0.08        32
           6       0.88      0.57      0.69       531

    accuracy                           0.87      3300
   macro avg       0.64      0.47      0.53      3300
weighted avg       0.87      0.87      0.86      3300

Random Classifier Accuracy:  87.30303030303031 %


In [ ]:
#Logistic Regression BoW
lreg_bow=LogisticRegression()
lreg_bow.fit(X_train_bow,y_train_bow)
y_pred_bow = lreg_bow.predict(X_test_bow)
report_bow = classification_report(y_test_bow, y_pred_bow)
acc = accuracy_score(y_test_bow,y_pred_bow)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report_bow)
print("Logistic Regression Accuracy: ", acc*100,"%")

              precision    recall  f1-score   support

           1       1.00      0.40      0.57        48
           2       0.99      0.67      0.80       302
           3       0.86      0.98      0.92      2386
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00        32
           6       0.82      0.57      0.67       531

    accuracy                           0.86      3300
   macro avg       0.61      0.43      0.49      3300
weighted avg       0.86      0.86      0.85      3300

Logistic Regression Accuracy:  86.45454545454545 %


In [ ]:
#SVM BOW
clf_svm = SVC(verbose=True)
clf_svm.fit(X_train_bow,y_train_bow)
y_pred_bow = clf_svm.predict(X_test_bow)
report_svm = classification_report(y_test_bow,y_pred_bow)
# print(report_svm)
acc_svm = accuracy_score(y_test_bow,y_pred_bow)

[LibSVM]

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report_svm)
print("SVM Accuracy: ", acc_svm*100,"%")

              precision    recall  f1-score   support

           1       1.00      0.40      0.57        48
           2       1.00      0.55      0.71       302
           3       0.80      1.00      0.89      2386
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00        32
           6       0.97      0.27      0.42       531

    accuracy                           0.82      3300
   macro avg       0.63      0.37      0.43      3300
weighted avg       0.84      0.82      0.78      3300

SVM Accuracy:  82.0909090909091 %


**Finding Accuracy with TF-IDF**

In [ ]:
X_train_tfidf.shape, X_test_tfidf.shape, y_train_tfidf.shape, y_test_tfidf.shape

((6700, 1451), (3300, 1451), (6700,), (3300,))

In [ ]:
#Applying Random Forest in TFIDF
classifier_tfidf =  RandomForestClassifier(n_estimators=50, verbose=True)
classifier_tfidf.fit(X_train_tfidf, y_train_tfidf)
y_pred_tfidf = classifier_tfidf.predict(X_test_tfidf)
report_tfidf = classification_report(y_test_tfidf, y_pred_tfidf)
acc = accuracy_score(y_test_tfidf,y_pred_tfidf) 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.1s finished
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report_tfidf)
print("Random Classifier Accuracy: ", acc*100,"%")

              precision    recall  f1-score   support

           1       1.00      0.44      0.61        48
           2       0.91      0.72      0.80       302
           3       0.85      0.95      0.90      2386
           4       0.00      0.00      0.00         1
           5       0.20      0.03      0.05        32
           6       0.72      0.49      0.59       531

    accuracy                           0.84      3300
   macro avg       0.61      0.44      0.49      3300
weighted avg       0.83      0.84      0.83      3300

Random Classifier Accuracy:  84.15151515151516 %


In [ ]:
#Logistic Regression TFIDF
lreg_tfidf=LogisticRegression()
lreg_tfidf.fit(X_train_tfidf,y_train_tfidf)
y_pred_tfidf = lreg_tfidf.predict(X_test_tfidf)
report_tfidf = classification_report(y_test_tfidf, y_pred_tfidf)
acc = accuracy_score(y_test_tfidf,y_pred_tfidf)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report_tfidf)
print("Logistic Regression Accuracy: ", acc*100,"%")

              precision    recall  f1-score   support

           1       1.00      0.33      0.50        48
           2       1.00      0.61      0.76       302
           3       0.83      0.98      0.90      2386
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00        32
           6       0.83      0.41      0.55       531

    accuracy                           0.84      3300
   macro avg       0.61      0.39      0.45      3300
weighted avg       0.84      0.84      0.82      3300

Logistic Regression Accuracy:  83.87878787878788 %


In [ ]:
#SVM TFIDF
clf_svm_tfidf = SVC(verbose=True)
clf_svm_tfidf.fit(X_train_tfidf,y_train_tfidf)
y_pred_tfidf = clf_svm_tfidf.predict(X_test_tfidf)
report_svm_tfidf = classification_report(y_test_tfidf,y_pred_tfidf)
# print(report_svm)
acc_svm_tfidf = accuracy_score(y_test_tfidf,y_pred_tfidf)


[LibSVM]

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(report_svm_tfidf)
print("Logistic Regression Accuracy: ", acc_svm_tfidf*100,"%")

              precision    recall  f1-score   support

           1       1.00      0.42      0.59        48
           2       1.00      0.63      0.77       302
           3       0.82      0.99      0.90      2386
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00        32
           6       0.90      0.35      0.50       531

    accuracy                           0.84      3300
   macro avg       0.62      0.40      0.46      3300
weighted avg       0.84      0.84      0.81      3300

Logistic Regression Accuracy:  83.75757575757575 %


In [ ]:
# Creating Pickle File
#classifier
file = "transform.pkl"
fileobj = open(file,'wb')
pickle.dump(classifier, fileobj)
fileobj.close()